# Q1: SIFT-BoVW-SVM

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
import os
import multiprocessing

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device,torch.__version__,torch.cuda.device_count()

In [ ]:
# GPU operations have a separate seed we also want to set
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Step 1: Load MNIST dataset
mnist_train = datasets.MNIST(root='./data', train=True, download=True)
mnist_test = datasets.MNIST(root='./data', train=False, download=True)

X_train, X_val, y_train, y_val = train_test_split(mnist_train.data.numpy(), mnist_train.targets.numpy(), test_size=0.01, random_state=42)
X_test, y_test = mnist_test.data.numpy(), mnist_test.targets.numpy()

In [ ]:
np.random.seed(42)
random_integers = [np.random.randint(0, 50000) for _ in range(4)]
fig, axes = plt.subplots(1, 4, figsize=(15, 4))
for i in range(4):
    axes[i].imshow(X_train[random_integers[i]], cmap="gray")
    axes[i].set_title(f"Label: {y_train[random_integers[i]]}")
    axes[i].axis('off')
plt.show()

## 1) SIFT detector and descriptor

In [ ]:
all_descriptors = []
descriptors_file = "./q1_pickle_files/all_descriptors.pkl"
if os.path.exists(descriptors_file):
    with open(descriptors_file,"rb") as f:
        all_descriptors = pickle.load(f)
else:
    for i in range(len(X_train)):
        keypoints, descriptors = cv2.SIFT_create().detectAndCompute(X_train[i], None)
        if descriptors is not None:
            all_descriptors.extend(descriptors)
    with open(descriptors_file,"wb") as f:
        pickle.dump(all_descriptors,f)

### K-means clustering of descriptors

In [ ]:
num_clusters = 100

if os.path.exists("./q1_pickle_files/kmeans_model.pkl"):
    with open('./q1_pickle_files/kmeans_model.pkl', 'rb') as f:
        kmeans = pickle.load(f)
else:
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(np.array(all_descriptors))
    with open('./q1_pickle_files/kmeans_model.pkl','wb') as f:
        pickle.dump(kmeans,f)

In [ ]:
kmeans.cluster_centers_.shape,kmeans.cluster_centers_

### Representing Images as Histograms

In [ ]:
def image_histogram(image, kmeans,hyp=None):
    if not hyp:
        keypoints, descriptors =  hyp.detectAndCompute(image,None)    
    else:
        keypoints, descriptors =  cv2.SIFT_create().detectAndCompute(image,None)
    if descriptors is not None:
        words = kmeans.predict(descriptors)
        histogram, _ = np.histogram(words, bins=range(len(kmeans.cluster_centers_)+1))
        return histogram
    else:
        return np.zeros(len(kmeans.cluster_centers_))

In [ ]:
pickle_file_train = './q1_pickle_files/X_train_bovw.pkl'
pickle_file_test = './q1_pickle_files/X_test_bovw.pkl'

if os.path.exists(pickle_file_train) and os.path.exists(pickle_file_test):
    with open(pickle_file_train, 'rb') as f:
        X_train_bovw = pickle.load(f)
    with open(pickle_file_test, 'rb') as f:
        X_test_bovw = pickle.load(f)
else:
    X_train_bovw = np.array([image_histogram(img, kmeans) for img in X_train])
    X_test_bovw = np.array([image_histogram(img, kmeans) for img in X_test])

    with open(pickle_file_train, 'wb') as f:
        pickle.dump(X_train_bovw, f)
    with open(pickle_file_test, 'wb') as f:
        pickle.dump(X_test_bovw, f)

### Linear SVM model

In [ ]:
if os.path.exists("./q1_pickle_files/svm_model.pkl"):
    with open('./q1_pickle_files/svm_model.pkl', 'rb') as f:
        svm = pickle.load(f)
else:
    svm = SVC(kernel='linear',random_state=42)
    svm.fit(X_train_bovw, y_train)
    with open('./q1_pickle_files/svm_model.pkl', 'wb') as f:
        pickle.dump(svm,f)

In [ ]:
y_pred = svm.predict(X_test_bovw)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

## 2) Changing no of clusters

In [ ]:
clusters = [10,30,100,500,1000]
accuracies = []
accuracies_file = "./q1_pickle_files/accuracies.pkl"
if os.path.exists(accuracies_file):
    with open(accuracies_file,"rb") as f:
        accuracies = pickle.load(f)
else:
    for num_clusters in clusters:
        
        ## kmeans_Clustering
        kmeans_file =f"./q1_pickle_files/kmeans_model_{num_clusters}.pkl"
        if os.path.exists(kmeans_file):
            with open(kmeans_file, 'rb') as f:
                kmeans = pickle.load(f)
        else:
            kmeans = KMeans(n_clusters=num_clusters, random_state=42)
            kmeans.fit(np.array(all_descriptors))
            with open(kmeans_file,'wb') as f:
                pickle.dump(kmeans,f)
    
        ## Representing using bag of words
        pickle_file_train = f'./q1_pickle_files/X_train_bovw_{num_clusters}.pkl'
        pickle_file_test = f'./q1_pickle_files/X_test_bovw{num_clusters}.pkl'
        
        if os.path.exists(pickle_file_train) and os.path.exists(pickle_file_test):
            with open(pickle_file_train, 'rb') as f:
                X_train_bovw = pickle.load(f)
            with open(pickle_file_test, 'rb') as f:
                X_test_bovw = pickle.load(f)
        else:
            X_train_bovw = np.array([image_histogram(img, kmeans) for img in X_train])
            X_test_bovw = np.array([image_histogram(img, kmeans) for img in X_test])
    
            # num_cores = 20  # Adjust this to the desired number of cores
            
            # with multiprocessing.Pool(num_cores) as pool:
            #     X_train_bovw = np.array(pool.map(image_histogram, X_train, [kmeans] * len(X_train)))
            #     X_test_bovw = np.array(pool.map(image_histogram, X_test, [kmeans] * len(X_test)))
    
            with open(pickle_file_train, 'wb') as f:
                pickle.dump(X_train_bovw, f)
            with open(pickle_file_test, 'wb') as f:
                pickle.dump(X_test_bovw, f)
    
    
        ## Training SVM Model
        if os.path.exists(f"./q1_pickle_files/svm_model_{num_clusters}.pkl"):
            with open(f'./q1_pickle_files/svm_model_{num_clusters}.pkl', 'rb') as f:
                svm = pickle.load(f)
        else:
            svm = SVC(kernel='linear',random_state=42)
            svm.fit(X_train_bovw, y_train)
            with open(f'./q1_pickle_files/svm_model_{num_clusters}.pkl', 'wb') as f:
                pickle.dump(svm,f)
    
        ## Storing Accuracies
        print(num_clusters)
        y_pred = svm.predict(X_test_bovw)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy_{num_clusters}:", accuracy)
        accuracies.append(accuracy)
    with open(accuracies_file,"wb") as f:
        pickle.dump(accuracies,f)

In [ ]:
for i in range(len(clusters)):
    print(clusters[i]," ==> ",accuracies[i])

In [ ]:
fig,ax = plt.subplots()
ax.plot(clusters,accuracies)
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Accuracy")
ax.set_title("No of clusters vs Accuracy Plot");

###### SIFT detector:
1. n_octaves: Higher values might capture keypoints at more scales, potentially improving accuracy for objects at different sizes or orientations, but could also increase computation time.
2. initial_sigma: Larger values could lead to more blurred features, affecting matching accuracy.
3. threshold: More aggressive filtering (lower values) might remove valid keypoints, reducing accuracy, while less filtering might increase false positives.

###### Linear SVM:
1. C: Higher values correspond to stronger regularization, potentially preventing overfitting but also reducing model flexibility.
2. loss: 'hinge' is the standard SVM loss, while 'squared_hinge' might be less sensitive to outliers but may have slower convergence.
3. tol: Tighter tolerance (lower values) might require more iterations during training but could lead to more precise models.

In [ ]:
# Define hyperparameter ranges to explore
sift_params = {
    'nfeatures': [50, 100, 200, 300],
    'nOctaveLayers': [1, 2, 3],
    'contrastThreshold': [0.03, 0.04, 0.05],
    'edgeThreshold': [10],  # Keep default value
    'sigma': [1.4, 1.6, 1.8],
    'enable_precise_upscale': [False]  # Keep default value
}

# Define hyperparameter ranges to explore
svm_params = {
    'C': [0.01, 0.1, 1.0, 10.0, 100.0],
    'loss': ['hinge', 'squared_hinge'],
    'tol': [1e-4, 1e-5],
}


In [ ]:
sift_params_file="./q1_pickle_files/random_sift_params.pkl"
svm_params_file="./q1_pickle_files/randon_svm_params.pkl"

# Check if pickle files exist
if not os.path.exists(sift_params_file):
    # Create and save random_sift_params
    random_sift_params = [{key: random.choice(value) for key, value in sift_params.items()} for _ in range(6)]
    with open(sift_params_file, 'wb') as f:
        pickle.dump(random_sift_params, f)
    print(f"{sift_params_file} created.")
else:
    with open(sift_params_file, 'rb') as f:
        random_sift_params = pickle.load(f)

if not os.path.exists(svm_params_file):
    # Create and save random_svm_params
    random_svm_params = [{key: random.choice(value) for key, value in svm_params.items()} for _ in range(6)]
    with open(svm_params_file, 'wb') as f:
        pickle.dump(random_svm_params, f)
    print(f"{svm_params_file} created.")
else:
    with open(svm_params_file, 'rb') as f:
        random_svm_params = pickle.load(f)

In [ ]:
accuracies_hyp=[]
accuracies_hyp_file ="./q1_pickle_files/accuracies_hyp.pkl"

if os.path.exists(accuracies_hyp_file):
    with open(accuracies_hyp_file,"rb") as f:
        accuracies_hyp = pickle.load(f)
else:
    for param_count in range(6):
    
        # calculating all descriptors
        sift = cv2.SIFT_create(**random_sift_params[param_count])
    
        all_descriptors = []
        descriptors_file = f"./q1_pickle_files/all_descriptors_{param_count}.pkl"
        if os.path.exists(descriptors_file):
            with open(descriptors_file,"rb") as f:
                all_descriptors = pickle.load(f)
        else:
            for i in range(len(X_train)):
                keypoints, descriptors = sift.detectAndCompute(X_train[i], None)
                if descriptors is not None:
                    all_descriptors.extend(descriptors)
            with open(descriptors_file,"wb") as f:
                pickle.dump(all_descriptors,f)
    
        # clustering using Kmeans
        num_clusters = 100
        
        if os.path.exists(f"./q1_pickle_files/kmeans_model_{param_count}.pkl"):
            with open(f'./q1_pickle_files/kmeans_model_{param_count}.pkl', 'rb') as f:
                kmeans = pickle.load(f)
        else:
            kmeans = KMeans(n_clusters=num_clusters, random_state=42)
            kmeans.fit(np.array(all_descriptors))
            with open(f'./q1_pickle_files/kmeans_model_{param_count}.pkl','wb') as f:
                pickle.dump(kmeans,f)
    
        #Representing images as histograms
    
        pickle_file_train = f'./q1_pickle_files/X_train_bovw_{param_count}.pkl'
        pickle_file_test = f'./q1_pickle_files/X_test_bovw_{param_count}.pkl'
        
        if os.path.exists(pickle_file_train) and os.path.exists(pickle_file_test):
            with open(pickle_file_train, 'rb') as f:
                X_train_bovw = pickle.load(f)
            with open(pickle_file_test, 'rb') as f:
                X_test_bovw = pickle.load(f)
        else:
            X_train_bovw = np.array([image_histogram(img, kmeans,sift) for img in X_train])
            X_test_bovw = np.array([image_histogram(img, kmeans,sift) for img in X_test])
        
            with open(pickle_file_train, 'wb') as f:
                pickle.dump(X_train_bovw, f)
            with open(pickle_file_test, 'wb') as f:
                pickle.dump(X_test_bovw, f)
    
        # Linear SVM Model
        if os.path.exists(f"./q1_pickle_files/svm_model_{param_count}.pkl"):
            with open(f'./q1_pickle_files/svm_model_{param_count}.pkl', 'rb') as f:
                svm = pickle.load(f)
        else:
            svm = SVC(kernel='linear',random_state=42)
            svm.fit(X_train_bovw, y_train)
            with open(f'./q1_pickle_files/svm_model_{param_count}.pkl', 'wb') as f:
                pickle.dump(svm,f)
        
        y_pred = svm.predict(X_test_bovw)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"SIFT params: {random_sift_params[param_count]}, SVM params: {random_svm_params[param_count]}, Accuracy: {accuracy}")
        accuracies_hyp.append(accuracy)
    with open(accuracies_hyp_file,"wb") as f:
        pickle.dump(accuracies_hyp,f)

In [ ]:
accuracies_hyp

In [ ]:
for param_count in range(6):
    print(f"SIFT params: {random_sift_params[param_count]}, SVM params: {random_svm_params[param_count]}, Accuracy: {accuracy}")
    print(accuracies_hyp[param_count])
    print("="*8)